In [1]:
from typing import Union
import ezdxf
from ezdxf.entities import Text, MText, Line, LWPolyline
from shapely.geometry import Point, LineString, Polygon, MultiPolygon
from shapely.ops import unary_union
from pathlib import Path
from itertools import chain

In [3]:
def extract_data_from_taxation_plan(file_path: Path,
                                    numbers_layers: list[str] = ["номера"],
                                    lines_layers: list[str] = ["полосы"],
                                    contours_layers: list[str] = ["контуры"],
                                    zones_layers: list[str] = ["зоны"]) -> tuple[list, list, list, list]:
    """
    Извлечение данных из чертежа таксации
    :param file_path: путь к файлу
    :param numbers_layers: слои с номерами
    :param lines_layers: слои с линиями
    :param contours_layers: слои с контурами
    :param zones_layers: слои с зонами
    :return: список объектов
    """

    doc = ezdxf.readfile(file_path)

    numbers, lines, contours, zones = [], [], [], []
    
    for entity in doc.modelspace():

        if isinstance(entity, Text) and entity.dxf.layer in numbers_layers:
            numbers.append(entity)
        elif isinstance(entity, MText) and entity.dxf.layer in numbers_layers:
            numbers.append(entity)

        elif isinstance(entity, LWPolyline) and entity.dxf.layer in lines_layers:
            lines.append(entity)
        elif isinstance(entity, Line) and entity.dxf.layer in lines_layers:
            lines.append(entity)

        elif isinstance(entity, LWPolyline) and entity.dxf.layer in contours_layers:
            contours.append(entity)

        elif isinstance(entity, LWPolyline) and entity.dxf.layer in zones_layers:
            zones.append(entity)
        elif isinstance(entity, MText) and entity.dxf.layer in zones_layers:
            zones.append(entity)
        elif isinstance(entity, Text) and entity.dxf.layer in zones_layers:
            zones.append(entity)

    return numbers, lines, contours, zones


In [12]:
def autocad_data_to_dict(entity_numbers: list, entity_lines: list, entity_contours: list, entity_zones: list):
    """
    Упорядочивание данных из объектов autocad в словари.
    :param entity_numbers: Список текстовых объектов autocad со слоя "номера"
    :param entity_lines: Список линейных объектов autocad со слоя "полосы"
    :param entity_contours: Список линейных объектов autocad со слоя "контуры"
    :param entity_zones: Список текстовых и линейных объектов autocad со слоя "зоны"
    :return: 
    """
    
    numbers = dict()    # key: k_number; value: number
    numbers_position = dict()   # key: k_number; value: k_position
    
    for k_number, text in enumerate(entity_numbers):
        number = text.plain_text().replace('\n', ' ') if isinstance(text, MText) else text.plain_text()
        numbers[k_number] = number
        k_position = Point(text.dxf.insert[0], text.dxf.insert[1])
        numbers_position[k_number] = k_position
        
    shapes = dict() # key: k_shape; value: shape
    
    k_shape = 0
    for line in entity_lines:
        if isinstance(line, LWPolyline):
            shape = LineString([(float(x), float(y)) for x, y in list(line.vertices())])
            shapes[k_shape] = shape
            k_shape += 1
        if isinstance(line, Line):
            shape = LineString([(line.dxf.start.x, line.dxf.start.y), (line.dxf.end.x, line.dxf.end.y)])
            shapes[k_shape] = shape
            k_shape += 1
    for contour in entity_contours:
        shape = Polygon([(float(x), float(y)) for x, y in list(contour.vertices())])
        shapes[k_shape] = shape
        k_shape += 1
        
    numbers_from_shape = dict()  # key: k_shape; value: list[k_number]
    
    min_distance = 0.01
    
    for k_number, k_position in numbers_position.items():
        for k_shape, shape in shapes.items():
            if isinstance(shape, LineString):
                distance = shape.distance(k_position)
            elif isinstance(shape, Polygon):
                distance = k_position.distance(shape.exterior)
            else:
                print(f"[WARNING]\tType shape {type(shape)} not is instance of LineString or Polygon.")
                continue
            if distance < min_distance:
                if k_shape not in numbers_from_shape:
                    numbers_from_shape[k_shape] = list()
                numbers_from_shape[k_shape].append(k_number)
    
    zone_shapes = dict()    # key: k_zone; value: shape
    
    k_zone = 0
    for entity in entity_zones:
        if isinstance(entity, LWPolyline):
            zone_shapes[k_zone] = Polygon([(float(x), float(y)) for x, y in list(entity.vertices())])
            k_zone += 1
        elif isinstance(entity, Text) or isinstance(entity, MText):
            continue
        else:
            print(f"[WARNING]\tType shape {type(entity)} not is instance of LWPolyline, Text or MText.")

    zone_names = dict()     # key: k_zone_name; value: zone_name
    zones_from_zone_names = dict()  # key: k_zone_name; value: list[k_zone]
    
    k_zone_name = 0
    zone_names_temp_list = []
    for entity in entity_zones:
        if isinstance(entity, Text) or isinstance(entity, MText):
            zone_name = entity.plain_text()
            if zone_name not in zone_names_temp_list:
                zone_names[k_zone_name] = zone_name
                zone_names_temp_list.append(zone_name)
                k_zone_name += 1
            zone_name_position = Point(entity.dxf.insert[0], entity.dxf.insert[1])
            for k_zone, shape in zone_shapes.items():
                if zone_name_position.distance(shape.exterior) < min_distance:
                    if k_zone_name not in zones_from_zone_names:
                        zones_from_zone_names[k_zone_name] = list()
                    zones_from_zone_names[k_zone_name].append(k_zone)
        elif isinstance(entity, LWPolyline):
            continue
        else:
            print(f"[WARNING]\tType shape {type(entity)} not is instance of LWPolyline, Text or MText.")
    tree_from_numbers = dict()
    shape_numbers_temp_list = []
    for k_shape, k_number_list in numbers_from_shape.items():
        shape_numbers_temp_list.extend(k_number_list)

    for k_number, number_position in numbers_position.items():
        if k_number not in shape_numbers_temp_list:
            tree_from_numbers[k_number] = number_position

    
    from pprint import pprint
    print("numbers")
    pprint(numbers)
    print("\n\nnumbers_position")
    pprint(numbers_position)
    print("\n\nshapes")
    pprint(shapes)
    print("\n\nnumbers_from_shape")
    pprint(numbers_from_shape)
    print("\n\nzone_shapes")
    pprint(zone_shapes)
    print("\n\nzone_names")
    pprint(zone_names)
    print("\n\nzones_from_zone_names")
    pprint(zones_from_zone_names)
    print("\n\ntree_from_numbers")
    pprint(tree_from_numbers)

In [13]:
dxf_data = extract_data_from_taxation_plan("test/test.dxf")
result = autocad_data_to_dict(*dxf_data)

numbers
{0: '1',
 1: '2',
 2: '3',
 3: '4',
 4: '5',
 5: '6',
 6: '7',
 7: '10',
 8: '11',
 9: '12',
 10: '14',
 11: '13',
 12: '8',
 13: '9',
 14: '15',
 15: '16',
 16: '17',
 17: '18',
 18: '19а-г',
 19: '20',
 20: '21',
 21: '22',
 22: '23',
 23: '24-25',
 24: '26-27',
 25: '28',
 26: '28',
 27: '28',
 28: '16'}


numbers_position
{0: <POINT (3075.966 1696.605)>,
 1: <POINT (3074.823 1693.901)>,
 2: <POINT (3108.838 1707.285)>,
 3: <POINT (3114.151 1707.47)>,
 4: <POINT (3115.691 1707.587)>,
 5: <POINT (3115.1 1706.181)>,
 6: <POINT (3114.7 1705.091)>,
 7: <POINT (3104.044 1698.544)>,
 8: <POINT (3105.091 1695.948)>,
 9: <POINT (3106.333 1693.701)>,
 10: <POINT (3104.361 1692.888)>,
 11: <POINT (3107.489 1689.421)>,
 12: <POINT (3106.666 1703.063)>,
 13: <POINT (3104.13 1690.619)>,
 14: <POINT (3092.269 1693.404)>,
 15: <POINT (3099.275 1706.814)>,
 16: <POINT (3103.397 1686.424)>,
 17: <POINT (3111.38 1713.542)>,
 18: <POINT (3115.877 1710.226)>,
 19: <POINT (3115.006 1715.725)>,
 

In [38]:
SPLIT_NUMBERS = ["1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11", "12", "13", "14", "15", "16", "17",
                 "18", "19а", "19б", "19в", "19г", "20", "21", "22", "23", "24", "25", "26", "27", "28"]
NUMBER_FROM_GET_KEY_FOR_SPLIT_NUMBER = ["1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11", "12", "13", "14",
                                        "15", "16", "17", "18", "19а-г", "19а-г", "19а-г", "19а-г", "20", 
                                        "21", "22", "23", "24-25", "24-25", "26-27", "26-27", "28"]
print(len(NUMBER_FROM_GET_KEY_FOR_SPLIT_NUMBER), len(SPLIT_NUMBERS))
list(zip(SPLIT_NUMBERS, NUMBER_FROM_GET_KEY_FOR_SPLIT_NUMBER))

31 31


[('1', '1'),
 ('2', '2'),
 ('3', '3'),
 ('4', '4'),
 ('5', '5'),
 ('6', '6'),
 ('7', '7'),
 ('8', '8'),
 ('9', '9'),
 ('10', '10'),
 ('11', '11'),
 ('12', '12'),
 ('13', '13'),
 ('14', '14'),
 ('15', '15'),
 ('16', '16'),
 ('17', '17'),
 ('18', '18'),
 ('19а', '19а-г'),
 ('19б', '19а-г'),
 ('19в', '19а-г'),
 ('19г', '19а-г'),
 ('20', '20'),
 ('21', '21'),
 ('22', '22'),
 ('23', '23'),
 ('24', '24-25'),
 ('25', '24-25'),
 ('26', '26-27'),
 ('27', '26-27'),
 ('28', '28')]